In [1]:
import numpy as np
import pandas as pd
tr = pd.read_csv('train2.csv')
#te = pd.read_csv('../input/test.csv')
from nltk.corpus import stopwords

import numpy as np
# Downloading stopwords from nltk library.
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
stop = stopwords.words('english')
tr_test = pd.read_csv('test_without_labels2.csv')

[nltk_data] Downloading package stopwords to C:\Users\Ngomba
[nltk_data]     Litombe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Ngomba
[nltk_data]     Litombe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
tr_test.head(5)

,Id,Question1,Question2
0,283003,What can someone do if they've lost the wirele...,What is the best USB wireless mouse that can b...
1,283004,Why India need to elect Prime minister?,Is prime minister of India elected or appointed?
2,283005,How can I make money online with free of cost?,How can I make money online for free?
3,283006,Does MDMA affect the first and higher order mo...,Do antipsychotics affect the first and higher ...
4,283007,"I am a Saudi National and have ""SR 3 million"" ...",Where should I invest money to get high returns?


In [3]:
tr_test.Question1.shape

(121287,)

In [4]:
def word_match_share(x):
    stops = set(stopwords.words('english'))
    q1 = x.Question1.fillna(' ').str.lower().str.split()
    q2 = x.Question2.fillna(' ').str.lower().str.split()
    q1 = q1.map(lambda l : set(l) - stops)
    q2 = q2.map(lambda l : set(l) - stops)
    q = pd.DataFrame({'q1':q1, 'q2':q2})
    q['len_inter'] = q.apply(lambda row : len(row['q1'] & row['q2']), axis=1)
    q['len_tot'] = q.q1.map(len) + q.q2.map(len)
    return (2 * q.len_inter / q.len_tot).fillna(0)

In [5]:
tr['word_share'] = word_match_share(tr)

In [6]:
tr_test['word_share'] = word_match_share(tr_test)

In [8]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(tr.IsDuplicate,tr.word_share.round())

In [7]:
import xgboost

In [11]:
import pandas as pd
from nltk.tokenize import word_tokenize
#importing dataset
dataset = tr
dataset= dataset.fillna('')
article=pd.concat([dataset['Question1'],dataset['Question2']])
article= article.apply(lambda x: " ".join(x.lower() for x in x.split())) #lowercase
article= article.str.replace('[^\w\s]','') #remove punctuation
article = article.str.replace('\d+', '') #remove numbers 

In [ ]:
#for test set
data_test = tr_test
data_test= data_test.fillna('')
article1=pd.concat([data_test['Question1'],data_test['Question2']])
article1= article1.apply(lambda x: " ".join(x.lower() for x in x.split())) #lowercase
article1= article1.str.replace('[^\w\s]','') #remove punctuation
article1 = article1.str.replace('\d+', '') #remove numbers 
#vectors2=vectorizer.transform(article1)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=1000)
vectorizer.fit(article)
vectors1 = vectorizer.transform(article)
vectors2 = vectorizer.transform(article1)

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
cm=[]
for i in range(283004):
    cm.append(cosine_similarity(vectors1[i],vectors1[283004+i]))

In [31]:
cm_test=[]
for i in range(tr_test.shape[0]):
    cm_test.append(cosine_similarity(vectors2[i],vectors2[121287+i]))

In [17]:
df = pd.DataFrame()
df['cos']=[k[0][0] for k in cm]
df['word_share']=tr.word_share
df1=df.to_numpy()

In [32]:
df2 = pd.DataFrame()
df2['cos']=[k[0][0] for k in cm_test]
df2['word_share']=tr_test.word_share
xtest=df2.to_numpy()

In [30]:
tr_test.shape[0]

121287

In [42]:
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report as clrp
kf = KFold(n_splits=5)

from xgboost import XGBClassifier
classifier2 = XGBClassifier()
import numpy as np


for train_index, test_index in kf.split(df1,tr.IsDuplicate):
    classifier2.fit(df1[train_index],tr.IsDuplicate[train_index])
    ypred=classifier2.predict(df1[test_index])
    ytestt=tr.IsDuplicate[test_index]
    print(clrp(ytestt,ypred))

              precision    recall  f1-score   support

           0       0.85      0.70      0.77     35456
           1       0.62      0.80      0.70     21145

    accuracy                           0.74     56601
   macro avg       0.74      0.75      0.73     56601
weighted avg       0.77      0.74      0.74     56601

              precision    recall  f1-score   support

           0       0.85      0.70      0.77     35675
           1       0.61      0.80      0.69     20926

    accuracy                           0.74     56601
   macro avg       0.73      0.75      0.73     56601
weighted avg       0.76      0.74      0.74     56601

              precision    recall  f1-score   support

           0       0.85      0.70      0.77     35431
           1       0.62      0.80      0.70     21170

    accuracy                           0.74     56601
   macro avg       0.74      0.75      0.73     56601
weighted avg       0.77      0.74      0.74     56601

              preci

In [33]:
classifier2.fit(df1,tr.IsDuplicate)
ypred = classifier2.predict(xtest)

In [34]:
naive = pd.DataFrame()
naive['Id'] = tr_test['Id']
naive['Predicted'] = ypred

In [89]:
naive.to_csv('naive.csv',index=False)